**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

In [1]:
go get -v github.com/aquasecurity/kube-bench

go: downloading github.com/aquasecurity/kube-bench v0.6.5
go: downloading github.com/aws/aws-sdk-go v1.40.51
go: downloading github.com/fatih/color v1.13.0
go: downloading github.com/golang/glog v0.0.0-20160126235308-23def4e6c14b
go: downloading github.com/spf13/cobra v1.2.1
go: downloading github.com/spf13/viper v1.9.0
go: downloading gorm.io/driver/postgres v1.1.1
go: downloading gorm.io/gorm v1.21.15
go: downloading github.com/onsi/ginkgo v1.16.4
go: downloading gopkg.in/yaml.v2 v2.4.0
go: downloading k8s.io/client-go v0.22.2
go: downloading github.com/pkg/errors v0.9.1
go: downloading github.com/mattn/go-colorable v0.1.9
go: downloading github.com/mattn/go-isatty v0.0.14
go: downloading github.com/inconshreveable/mousetrap v1.0.0
go: downloading github.com/spf13/pflag v1.0.5
go: downloading github.com/fsnotify/fsnotify v1.5.1
go: downloading github.com/magiconair/properties v1.8.5
go: downloading github.com/mitchellh/mapstructure v1.4.2
go: downloading github.com/spf13/afero v1.6.0

In [2]:
kube-bench


unable to determine benchmark version: config file is missing 'version_mapping' section


: 1

In [11]:
git clone https://github.com/aquasecurity/kube-bench

Cloning into 'kube-bench'...
remote: Enumerating objects: 4115, done.        
remote: Counting objects: 100% (721/721), done.        
remote: Compressing objects: 100% (360/360), done.        
remote: Total 4115 (delta 435), reused 545 (delta 313), pack-reused 3394        
Receiving objects: 100% (4115/4115), 8.13 MiB | 2.80 MiB/s, done.
Resolving deltas: 100% (2586/2586), done.


In [12]:
cd kube-bench

In [13]:
ls

CONTRIBUTING.md  cfg            go.mod       job-ack.yaml       job-master.yaml
Dockerfile       check          go.sum       job-aks.yaml       job-node.yaml
LICENSE          cmd            hack         job-eks-asff.yaml  job.yaml
NOTICE           codecov.yml    hooks        job-eks.yaml       main.go
OWNERS           docs           integration  job-gke.yaml       makefile
README.md        entrypoint.sh  internal     job-iks.yaml       mkdocs.yml


In [14]:
kube-bench

[INFO] 1 Master Node Security Configuration
[INFO] 1.1 Master Node Configuration Files
[PASS] 1.1.1 Ensure that the API server pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.2 Ensure that the API server pod specification file ownership is set to root:root (Automated)
[PASS] 1.1.3 Ensure that the controller manager pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.4 Ensure that the controller manager pod specification file ownership is set to root:root (Automated)
[PASS] 1.1.5 Ensure that the scheduler pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.6 Ensure that the scheduler pod specification file ownership is set to root:root (Automated)
[PASS] 1.1.7 Ensure that the etcd pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.8 Ensure that the etcd pod specification file ownership is set to root:root (Automated)
[WA

In [5]:
kube-bench --help

This tool runs the CIS Kubernetes Benchmark (https://www.cisecurity.org/benchmark/kubernetes/)

Usage:
  kube-bench [flags]
  kube-bench [command]

Available Commands:
  help        Help about any command
  master      Run Kubernetes benchmark checks from the master.yaml file.
  node        Run Kubernetes benchmark checks from the node.yaml file.
  run         Run tests
  version     Shows the version of kube-bench.

Flags:
      --alsologtostderr                  log to standard error as well as files
      --asff                             Send the results to AWS Security Hub
      --benchmark string                 Manually specify CIS benchmark version. It would be an error to specify both --version and --benchmark flags
  -c, --check string                     A comma-delimited list of checks to run as specified in CIS document. Example --check="1.1.1,1.1.2"
      --config string                    config file (default is ./cfg/config.yaml)
  -D, --config-dir string              

In [15]:
ls

CONTRIBUTING.md  cfg            go.mod       job-ack.yaml       job-master.yaml
Dockerfile       check          go.sum       job-aks.yaml       job-node.yaml
LICENSE          cmd            hack         job-eks-asff.yaml  job.yaml
NOTICE           codecov.yml    hooks        job-eks.yaml       main.go
OWNERS           docs           integration  job-gke.yaml       makefile
README.md        entrypoint.sh  internal     job-iks.yaml       mkdocs.yml


In [16]:
ls cfg

ack-1.0  cis-1.20  cis-1.6      eks-1.0.1  rh-0.7
aks-1.0  cis-1.5   config.yaml  gke-1.0    rh-1.0


In [17]:
cat cfg/config.yaml

---
## Controls Files.
# These are YAML files that hold all the details for running checks.
#
## Uncomment to use different control file paths.
# masterControls: ./cfg/master.yaml
# nodeControls: ./cfg/node.yaml

master:
  components:
    - apiserver
    - scheduler
    - controllermanager
    - etcd
    - flanneld
    # kubernetes is a component to cover the config file /etc/kubernetes/config that is referred to in the benchmark
    - kubernetes
    - kubelet

  kubernetes:
    defaultconf: /etc/kubernetes/config

  apiserver:
    bins:
      - "kube-apiserver"
      - "hyperkube apiserver"
      - "hyperkube kube-apiserver"
      - "apiserver"
      - "openshift start master api"
      - "hypershift openshift-kube-apiserver"
    confs:
      - /etc/kubernetes/manifests/kube-apiserver.yaml
      - /etc/kubernetes/manifests/kube-apiserver.yml
      - /etc/kubernetes/manifests/kube-apiserver.manifest
      - /var/snap/kube-apiserver/current/args
      - /var/snap/microk8s/current/args/k

In [18]:
cat job-aks.yaml

---
apiVersion: batch/v1
kind: Job
metadata:
  name: kube-bench
spec:
  template:
    spec:
      hostPID: true
      containers:
        - name: kube-bench
          image: aquasec/kube-bench:latest
          command: ["kube-bench", "run", "--targets", "node", "--benchmark", "aks-1.0"]
          volumeMounts:
            - name: var-lib-kubelet
              mountPath: /var/lib/kubelet
              readOnly: true
            - name: etc-systemd
              mountPath: /etc/systemd
              readOnly: true
            - name: etc-default
              mountPath: /etc/default
              readOnly: true
            - name: etc-kubernetes
              mountPath: /etc/kubernetes
              readOnly: true
      restartPolicy: Never
      volumes:
        - name: var-lib-kubelet
          hostPath:
            path: "/var/lib/kubelet"
        - name: etc-systemd
          hostPath:
            path: "/etc/systemd"
        - name: etc-default
          hostPath:
            path: 

In [22]:
cd 

In [23]:
git clone https://github.com/kubernetes/kube-state-metrics

Cloning into 'kube-state-metrics'...
remote: Enumerating objects: 25754, done.        
remote: Counting objects: 100% (4608/4608), done.        
remote: Compressing objects: 100% (552/552), done.        
remote: Total 25754 (delta 4184), reused 4056 (delta 4056), pack-reused 21146        
Receiving objects: 100% (25754/25754), 20.20 MiB | 9.42 MiB/s, done.
Resolving deltas: 100% (16723/16723), done.


In [24]:
cd kube-state-metrics

In [26]:
make

- Verifying that the dependencies have expected content...
go mod verify
all modules verified
- Checking for any unused/missing packages in go.mod...
go mod tidy
go: downloading github.com/prometheus/client_golang v1.11.0
go: downloading k8s.io/autoscaler/vertical-pod-autoscaler v0.9.2
go: downloading k8s.io/client-go v0.22.4
go: downloading github.com/oklog/run v1.1.0
go: downloading github.com/prometheus/common v0.32.1
go: downloading github.com/prometheus/exporter-toolkit v0.7.0
go: downloading k8s.io/klog/v2 v2.30.0
go: downloading k8s.io/api v0.22.4
go: downloading k8s.io/apimachinery v0.22.4
go: downloading github.com/dgryski/go-jump v0.0.0-20211018200510-ba001c3ffce0
go: downloading github.com/prometheus/client_model v0.2.0
go: downloading github.com/brancz/gojsontoyaml v0.1.0
go: downloading github.com/campoy/embedmd v1.0.0
go: downloading github.com/google/go-jsonnet v0.17.0
go: downloading github.com/jsonnet-bundler/jsonnet-bundler v0.4.1-0.20200708074244-ada055a225fa
go: dow

In [27]:
ls

CHANGELOG.md     RELEASE.md          examples            main_test.go
CONTRIBUTING.md  SECURITY.md         go.mod              pkg
Dockerfile       SECURITY_CONTACTS   go.sum              scripts
LICENSE          VERSION             internal            tests
Makefile         cloudbuild.yaml     jsonnet             tools
OWNERS           code-of-conduct.md  kustomization.yaml
README.md        docs                main.go


In [30]:
go build

In [31]:
ls

CHANGELOG.md     RELEASE.md          examples            main.go
CONTRIBUTING.md  SECURITY.md         go.mod              main_test.go
Dockerfile       SECURITY_CONTACTS   go.sum              pkg
LICENSE          VERSION             internal            scripts
Makefile         cloudbuild.yaml     jsonnet             tests
OWNERS           code-of-conduct.md  kube-state-metrics  tools
README.md        docs                kustomization.yaml


In [33]:
./kube-state-metrics --help

Usage of ./kube-state-metrics:
      --add_dir_header                        If true, adds the file directory to the header of the log messages
      --alsologtostderr                       log to standard error as well as files
      --apiserver string                      The URL of the apiserver to use as a master
      --enable-gzip-encoding                  Gzip responses when requested by clients via 'Accept-Encoding: gzip' header.
  -h, --help                                  Print Help text
      --host string                           Host to expose metrics on. (default "::")
      --kubeconfig string                     Absolute path to the kubeconfig file
      --log_backtrace_at traceLocation        when logging hits line file:N, emit a stack trace (default :0)
      --log_dir string                        If non-empty, write log files in this directory
      --log_file string                       If non-empty, use this log file
      --log_file_max_size uint              

In [5]:
kubectl get pods -A

NAMESPACE              NAME                                        READY   STATUS    RESTARTS   AGE
kube-system            coredns-74ff55c5b-kxfll                     1/1     Running   0          4m22s
kube-system            etcd-minikube                               1/1     Running   0          4m32s
kube-system            kube-apiserver-minikube                     1/1     Running   0          4m32s
kube-system            kube-controller-manager-minikube            1/1     Running   0          4m32s
kube-system            kube-proxy-5d97p                            1/1     Running   0          4m22s
kube-system            kube-scheduler-minikube                     1/1     Running   0          4m32s
kube-system            storage-provisioner                         1/1     Running   0          4m37s
kubernetes-dashboard   dashboard-metrics-scraper-c95fcf479-5nrbr   1/1     Running   0          21s
kubernetes-dashboard   kubernetes-dashboard-6cff4c7c4f-tf2ws       1/1     Running   0

In [7]:
cd /root/go/src/github.com/kubernetes/kube-state-metrics

In [8]:
ls

CHANGELOG.md     RELEASE.md          docs                main.go
CONTRIBUTING.md  SECURITY.md         examples            main_test.go
Dockerfile       SECURITY_CONTACTS   go.mod              pkg
LICENSE          VERSION             go.sum              scripts
Makefile         charts              internal            tests
OWNERS           cloudbuild.yaml     jsonnet             tools
README.md        code-of-conduct.md  kustomization.yaml


In [9]:
make

- Verifying that the dependencies have expected content...
go mod verify
all modules verified
- Checking for any unused/missing packages in go.mod...
go mod tidy
go: downloading github.com/prometheus/common v0.15.0
go: downloading k8s.io/api v0.19.3
go: downloading github.com/prometheus/client_golang v1.7.1
go: downloading k8s.io/client-go v0.19.3
go: downloading github.com/oklog/run v1.1.0
go: downloading k8s.io/apimachinery v0.19.3
go: downloading github.com/prometheus/client_model v0.2.0
go: downloading github.com/robfig/cron/v3 v3.0.0
go: downloading github.com/beorn7/perks v1.0.1
go: downloading golang.org/x/oauth2 v0.0.0-20200107190931-bf48bf16ab8d
go: downloading github.com/prometheus/procfs v0.1.3
go: downloading golang.org/x/crypto v0.0.0-20200622213623-75b288015ac9
go: downloading github.com/imdario/mergo v0.3.5
go: downloading github.com/Azure/go-autorest/autorest v0.10.0
go: downloading k8s.io/autoscaler/vertical-pod-autoscaler v0.0.0-20200727194258-b7922d74509c
go: downloa

In [10]:
go install

In [11]:
ls

CHANGELOG.md     RELEASE.md          docs                main.go
CONTRIBUTING.md  SECURITY.md         examples            main_test.go
Dockerfile       SECURITY_CONTACTS   go.mod              pkg
LICENSE          VERSION             go.sum              scripts
Makefile         charts              internal            tests
OWNERS           cloudbuild.yaml     jsonnet             tools
README.md        code-of-conduct.md  kustomization.yaml


In [12]:
kube-state-metrics

I0120 14:33:07.939307   45601 main.go:96] Using default resources
I0120 14:33:07.939464   45601 main.go:108] Using all namespace
I0120 14:33:07.939501   45601 main.go:129] metric allow-denylisting: Excluding the following lists that were on denylist: 
W0120 14:33:07.939531   45601 client_config.go:608] Neither --kubeconfig nor --master was specified.  Using the inClusterConfig.  This might not work.
W0120 14:33:07.939544   45601 client_config.go:613] error creating inClusterConfig, falling back to default config: unable to load in-cluster configuration, KUBERNETES_SERVICE_HOST and KUBERNETES_SERVICE_PORT must be defined
F0120 14:33:07.939660   45601 main.go:139] Failed to create client: invalid configuration: no configuration has been provided, try setting KUBERNETES_MASTER environment variable
goroutine 1 [running]:
k8s.io/klog/v2.stacks(0xc000010001, 0xc0001b03c0, 0xb2, 0x1da)
	/root/go/pkg/mod/k8s.io/klog/v2@v2.2.0/klog.go:996 +0xb9
k8s.io/klog/v2.(*loggingT).output(0x21fe0a0, 0xc00

: 255

In [13]:
cat ~/.kube/config

apiVersion: v1
clusters:
- cluster:
    certificate-authority: /var/lib/minikube/certs/ca.crt
    server: https://localhost:8443
  name: ""
contexts:
- context:
    cluster: ""
    user: ""
  name: ""
current-context: ""
kind: Config
preferences: {}
users:
- name: ""
  user:
    client-certificate: /var/lib/minikube/certs/apiserver.crt
    client-key: /var/lib/minikube/certs/apiserver.key


In [19]:
kube-state-metrics --kubeconfig /root/.kube/config

I0121 15:53:02.276733  389321 main.go:96] Using default resources
I0121 15:53:02.278868  389321 main.go:108] Using all namespace
I0121 15:53:02.278882  389321 main.go:129] metric allow-denylisting: Excluding the following lists that were on denylist: 
I0121 15:53:02.286636  389321 main.go:235] Testing communication with server
I0121 15:53:02.303759  389321 main.go:240] Running with Kubernetes cluster version: v1.20. git version: v1.20.0. git tree state: clean. commit: af46c47ce925f4c4ad5cc8d1fca46c7b77d13b38. platform: linux/amd64
I0121 15:53:02.303806  389321 main.go:242] Communication with server successful
I0121 15:53:02.304670  389321 main.go:198] Starting metrics server: [::]:8080
I0121 15:53:02.304752  389321 metrics_handler.go:96] Autosharding disabled
I0121 15:53:02.304859  389321 main.go:187] Starting kube-state-metrics self metrics server: [::]:8081
I0121 15:53:02.305702  389321 builder.go:164] Active resources: certificatesigningrequests,configmaps,cronjobs,daemonsets,deploy

In [17]:
kube-state-metrics --help

Usage of kube-state-metrics:
      --add_dir_header                    If true, adds the file directory to the header of the log messages
      --alsologtostderr                   log to standard error as well as files
      --apiserver string                  The URL of the apiserver to use as a master
      --enable-gzip-encoding              Gzip responses when requested by clients via 'Accept-Encoding: gzip' header.
  -h, --help                              Print Help text
      --host string                       Host to expose metrics on. (default "::")
      --kubeconfig string                 Absolute path to the kubeconfig file
      --labels-metric-allow-list string   Allows to pass a list of additional Kubernetes label keys that will be used in the resource' labels metric. By default the metric contains only name and namespace labels. To include additional labels provide a list of resource names in their plural form and Kubernetes label keys you would like to allow for them 

In [18]:
curl minikube:8081

curl: (7) Failed to connect to minikube port 8081: Connection refused


: 7

In [ ]:
curl -s https://raw.githubusercontent.com/armosec/kubescape/master/install.sh | /bin/bash